# EX7

## Enunciado
A empresa **TUDOLIMPO** fabrica 3 tipos de amaciantes: **super**, **extra** e **comum**. O objetivo é determinar quanto se deve produzir de cada tipo de amaciante para **maximizar o lucro líquido**.

### Informações do problema:
- **Lucro líquido por kg de amaciante**:
  - Amaciante super: **R$5,00**
  - Amaciante extra: **R$4,00**
  - Amaciante comum: **R$3,00**

- **Tempos de processamento por operação** (em minutos por kg produzido):
  | Operação   | Amaciante Super | Amaciante Extra | Amaciante Comum |
  |------------|-----------------|-----------------|-----------------|
  | Operação 1 | 6               | 4               | 2               |
  | Operação 2 | 3               | 6               | 3               |
  | Operação 3 | 4               | 10              | 4               |

- **Tempo total disponível para cada operação**:
  | Operação   | Tempo disponível (minutos) |
  |------------|----------------------------|
  | Operação 1 | 240                        |
  | Operação 2 | 180                        |
  | Operação 3 | 200                        |

### Problema a ser resolvido
Quanto produzir de cada amaciante (**super**, **extra** e **comum**) para maximizar o lucro líquido, considerando o tempo disponível para cada operação?

---


###  Modelo Matemático

### Modelo Formal do Problema

#### Índices:
- $( i )$: Tipos amaciante, onde $( i \in \{ \text{Super}, \text{Extra}, \text{Comum} \} )$.
- $( j )$: operações de produção, onde $( j \in \{ \text{1}, \text{2}, \text{3} \} )$.

#### Parâmetros:
- $( t_{ij} )$: Tempo médio de operação (em minutos) para produzir um amaciante do modelo $( i )$ no operação $( j )$.
- $( T_j )$: Tempo disponível no operação $( j )$.
- $( L_i )$: Lucro unitário do tipo $( i )$.


#### Variáveis de Decisão:
- $( x_i )$: Quantidade de amaciante do tipo $( i )$ a serem fabricados.

#### Função Objetivo:
Maximizar o lucro :

$$
Z = \sum_{i} L_i \cdot x_i
$$

#### Restrições:

1. **Tempo disponível :**

$$
\sum_{i} t_{ij} \cdot x_i \leq T_j , \quad \forall j
$$


3. **Não negatividade:**

$$
x_i \geq 0, \quad \forall i
$$


In [10]:
import pyomo.environ as pyo

In [11]:
# Criação do modelo
model = pyo.ConcreteModel()

In [12]:

# Conjuntos
model.P = pyo.Set(initialize=['Super', 'Extra', 'Comum'])  # Modelos de carros
model.S = pyo.Set(initialize=['1', '2', '3'])  # operaçãoes

In [13]:
# ------------------------------
# Parâmetros
# ------------------------------
lucro = {'Super':5, 'Extra':4, 'Comum':3}
model.lucro = pyo.Param(model.P,initialize=lucro)


tempo_operacao = {
    ('Super', '1'): 6, ('Super', '2'): 3, ('Super', '3'): 4,
    ('Extra', '1'): 4, ('Extra', '2'): 6, ('Extra', '3'): 10,
    ('Comum', '1'): 2, ('Comum', '2'): 3, ('Comum', '3'): 4
} 
model.tempo_operacao = pyo.Param(model.P,model.S, initialize=tempo_operacao)

tempo_disponiveis = {'1': 240, '2': 180, '3': 200}
model.tempo_disponiveis = pyo.Param(model.S, initialize=tempo_disponiveis)



In [14]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals)  # Quantidade produzida de cada modelo


In [15]:

# ------------------------------
# Função Objetivo
# ------------------------------
def lucro_total(model):
    return sum(model.lucro[j] * model.x[j] for j in model.P)
model.objetivo = pyo.Objective(rule=lucro_total, sense=pyo.maximize)


In [16]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de capacidade por operação
def restricao_capacidade(model, i):
    return sum(model.tempo_operacao[j,i] * model.x[j] for j in model.P) <= model.tempo_disponiveis[i]
model.capacidade = pyo.Constraint(model.S, rule=restricao_capacidade)


In [17]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex7.lp", io_options={"symbolic_solver_labels": True})

('ex7.lp', 2414253997504)

In [18]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



LP   has 3 rows; 3 cols; 9 nonzeros
Coefficient ranges:
  Matrix [2e+00, 1e+01]
  Cost   [3e+00, 5e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 2e+02]
Presolving model
3 rows, 3 cols, 9 nonzeros  0s
3 rows, 3 cols, 9 nonzeros  0s
Presolve : Reductions: rows 3(-0); columns 3(-0); elements 9(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -3.2499955273e+00 Ph1: 3(10.25); Du: 3(3.25) 0s
          2     2.2000000000e+02 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  2.2000000000e+02
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.01

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 220,00


In [19]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto Super : 35.00
  Quantidade do produto Comum : 15.00
